In [205]:
import numpy as np

### QR Compression

Here's an example of how a conversion of a big tensor to an MPS form could go: we iteratively sweep doing QR decompositions 

In [206]:
def tensor_to_mps_qr(w):

    nspins = int(np.log2(np.size(w)))
    print(f"{nspins=}")

    mps_tensors = []
    # first tensor 
    w = np.reshape(w, (2,2**(nspins-1)))
    q,r = np.linalg.qr(w)
    mps_tensors.append(q)
    virtual_dim = np.shape(q)[1]
    print(f"{virtual_dim=}")

    for nn in range(2,nspins):
        print(f"Factorizing {nn}th tensor")
        #print(f"{np.shape(r)=} ")
        r = np.reshape(r,(virtual_dim*2,2**(nspins-nn)))
        #print(f"{np.shape(r)=}")
        q, r = np.linalg.qr(r)
        mps_tensors.append(np.reshape(q,(virtual_dim,2,np.shape(q)[1])))
        virtual_dim = np.shape(q)[1]
        print(f"{virtual_dim=}")

    mps_tensors.append(np.reshape(r, (virtual_dim, 2)))


    return mps_tensors

Let's check if this works: build a state for say 10 qubits, and construct the MPS tensors from it.

We can also check what the sizes of the matrices involved are, 
in principle the maximum bond dimension allowed is 2^(N/2)

In [207]:
w = (np.random.rand(1024) + 1j*np.random.rand(1024) )
w = w/np.linalg.norm(w)

mpsten = tensor_to_mps_qr(w)

len(mpsten)

for ii,ten in enumerate(mpsten):
    print(f"{ii} - {np.shape(ten)}")

nspins=10
virtual_dim=2
Factorizing 2th tensor
virtual_dim=4
Factorizing 3th tensor
virtual_dim=8
Factorizing 4th tensor
virtual_dim=16
Factorizing 5th tensor
virtual_dim=32
Factorizing 6th tensor
virtual_dim=16
Factorizing 7th tensor
virtual_dim=8
Factorizing 8th tensor
virtual_dim=4
Factorizing 9th tensor
virtual_dim=2
0 - (2, 2)
1 - (2, 2, 4)
2 - (4, 2, 8)
3 - (8, 2, 16)
4 - (16, 2, 32)
5 - (32, 2, 16)
6 - (16, 2, 8)
7 - (8, 2, 4)
8 - (4, 2, 2)
9 - (2, 2)


Try to reconstruct the original tensor from the MPS tensors, to make sure that we are doing things right

In [208]:
# Try to reconstruct 

def wten_from_mps(mps_list):

    # First tensor
    w_rec = mps_list[0]

    for jj in range(1,len(mps_list)-1):
        print(f"{jj} - {np.shape(w_rec)} * {np.shape(mps_list[jj])}")
        w_rec = np.einsum("ia,ajb->ijb", w_rec, mps_list[jj])
        w_rec = np.reshape(w_rec, (np.shape(w_rec)[0] * np.shape(w_rec)[1], np.shape(w_rec)[2]))

    # Final tensor 
    w_rec = np.einsum("ia,aj->ij", w_rec, mps_list[-1])
    w_rec = np.reshape(w_rec, (np.shape(w_rec)[0] * np.shape(w_rec)[1]))


    return w_rec

In [209]:
w_reconstructed = wten_from_mps(mpsten)

np.allclose(w, w_reconstructed)

1 - (2, 2) * (2, 2, 4)
2 - (4, 4) * (4, 2, 8)
3 - (8, 8) * (8, 2, 16)
4 - (16, 16) * (16, 2, 32)
5 - (32, 32) * (32, 2, 16)
6 - (64, 16) * (16, 2, 8)
7 - (128, 8) * (8, 2, 4)
8 - (256, 4) * (4, 2, 2)


True

The resulting matrices have a specific property: due to the way we did our sweep, 

the MPS will be in *left* canonical form. Let's check it 

In [210]:
np.einsum("pv,pw->vw", mpsten[0],mpsten[0].conj())

array([[ 1.00000000e+00-1.05480848e-17j, -5.55111512e-17+2.22044605e-16j],
       [-5.55111512e-17-2.22044605e-16j,  1.00000000e+00-4.92560725e-18j]])

In [211]:
np.einsum("lpr,lpk->rk", mpsten[1],mpsten[1].conj())

array([[ 1.00000000e+00+1.67544707e-17j,  5.55111512e-17+6.24500451e-17j,
         0.00000000e+00-9.97465999e-18j, -4.85722573e-17+0.00000000e+00j],
       [ 5.55111512e-17-6.24500451e-17j,  1.00000000e+00+8.17970144e-18j,
         0.00000000e+00+2.77555756e-17j, -1.38777878e-17+1.38777878e-16j],
       [ 0.00000000e+00+3.42607887e-17j,  0.00000000e+00-2.08166817e-17j,
         1.00000000e+00-3.08544155e-18j, -1.38777878e-17-2.77555756e-17j],
       [-4.85722573e-17+0.00000000e+00j, -1.38777878e-17-1.11022302e-16j,
        -1.38777878e-17+2.77555756e-17j,  1.00000000e+00-1.60777876e-17j]])

In [212]:
lld = np.einsum("lpr,lpk->rk", mpsten[3],mpsten[3].conj())
print(np.shape(lld))
np.allclose(lld, np.eye(np.shape(lld)[1]))

(16, 16)


True

### E1:  Factorization using SVD 

We can do something analogous using SVD. While QR is faster, SVD allows us to 
estimate the entanglement of the state, and will allow us to truncate in a controlled way.

Write a function analogous to `tensor_to_mps_qr()` which instead uses SVD to factorize the state


In [213]:
def tensor_to_mps_svd(w):

    nspins = int(np.log2(np.size(w)))

    print(f"{nspins=}")

    mps_tensors = []
    # first tensor 
    w = np.reshape(w, (2,2**(nspins-1)))
    u,s,vd = np.linalg.svd(w,full_matrices=False)
    mps_tensors.append(u)
    virtual_dim = np.shape(u)[1]
    print(f"{virtual_dim=}")
    s_vd = np.einsum("ab,bj->aj",np.diag(s),vd)

    for nn in range(2,nspins):
        print(f"Factorizing {nn}th tensor")
        #print(f"{np.shape(r)=} ")
        s_vd = np.reshape(s_vd,(virtual_dim*2,2**(nspins-nn)))
        #print(f"{np.shape(r)=}")
        u, s, vd = np.linalg.svd(s_vd,full_matrices=False)
        mps_tensors.append(np.reshape(u,(virtual_dim,2,np.shape(u)[1])))
        virtual_dim = np.shape(u)[1]
        print(f"{virtual_dim=}")
        s_vd = np.einsum("ab,bj->aj",np.diag(s),vd)


    mps_tensors.append(np.reshape(s_vd, (virtual_dim, 2)))


    return mps_tensors

Let's test whether this works 

In [214]:
w = (np.random.rand(1024) + 1j*np.random.rand(1024) )
w = w/np.linalg.norm(w)

mpsten = tensor_to_mps_svd(w)

len(mpsten)

for ii,ten in enumerate(mpsten):
    print(f"{ii} - {np.shape(ten)}")

w_reconstructed = wten_from_mps(mpsten)

np.allclose(w, w_reconstructed)

nspins=10
virtual_dim=2
Factorizing 2th tensor
virtual_dim=4
Factorizing 3th tensor
virtual_dim=8
Factorizing 4th tensor
virtual_dim=16
Factorizing 5th tensor
virtual_dim=32
Factorizing 6th tensor
virtual_dim=16
Factorizing 7th tensor
virtual_dim=8
Factorizing 8th tensor
virtual_dim=4
Factorizing 9th tensor
virtual_dim=2
0 - (2, 2)
1 - (2, 2, 4)
2 - (4, 2, 8)
3 - (8, 2, 16)
4 - (16, 2, 32)
5 - (32, 2, 16)
6 - (16, 2, 8)
7 - (8, 2, 4)
8 - (4, 2, 2)
9 - (2, 2)
1 - (2, 2) * (2, 2, 4)
2 - (4, 4) * (4, 2, 8)
3 - (8, 8) * (8, 2, 16)
4 - (16, 16) * (16, 2, 32)
5 - (32, 32) * (32, 2, 16)
6 - (64, 16) * (16, 2, 8)
7 - (128, 8) * (8, 2, 4)
8 - (256, 4) * (4, 2, 2)


True

In [215]:
v1 = [1,0.1,0.01,0.00001]
v1 /= np.linalg.norm(v1)

v1[v1 > 0.01]

array([0.99498793, 0.09949879])

In [216]:
def svd_trunc(m,cutoff=1e-14,maxdim=500):
    u, s, vd = np.linalg.svd(m,full_matrices=False)
    snorm = s/np.linalg.norm(s)
    cut = min(len(snorm[snorm > cutoff]), maxdim)
    print(f"{cut=}")
    print(f"Last SV = {snorm[-1]}")

    return u[:,:cut], s[:cut], vd[:cut,:]

svd_trunc(np.random.rand(1000,1000), cutoff=1e-4)


cut=500
Last SV = 7.96016412744189e-06


(array([[-0.03134922, -0.04192444,  0.01019647, ..., -0.00565838,
          0.02213023,  0.0366036 ],
        [-0.03176487, -0.00177647,  0.00019864, ...,  0.01507021,
         -0.0428183 ,  0.01533865],
        [-0.03141648, -0.01652561, -0.03680401, ..., -0.02713592,
         -0.0479172 , -0.00041648],
        ...,
        [-0.03115906, -0.01308556,  0.0301592 , ..., -0.01950727,
         -0.0629922 , -0.00266309],
        [-0.03157493, -0.05849522,  0.01041717, ..., -0.00277386,
         -0.03694582, -0.02716371],
        [-0.03129119, -0.02455811,  0.02962391, ..., -0.02021741,
          0.0418008 ,  0.06416723]]),
 array([500.07064274,  18.1534261 ,  18.03892091,  17.96823521,
         17.96104582,  17.81855864,  17.75486459,  17.6872707 ,
         17.6648036 ,  17.59961177,  17.56122444,  17.44835445,
         17.39318307,  17.33400271,  17.29847596,  17.25086915,
         17.21872288,  17.1533544 ,  17.10160437,  17.07694838,
         17.04105226,  16.98582467,  16.96670259,  16

In [217]:
def tensor_to_mps_svd_trunc(w):

    nspins = int(np.log2(np.size(w)))

    print(f"{nspins=}")

    mps_tensors = []
    # first tensor 
    w = np.reshape(w, (2,2**(nspins-1)))
    u,s,vd = svd_trunc(w)
    mps_tensors.append(u)
    virtual_dim = np.shape(u)[1]
    print(f"{virtual_dim=}")
    s_vd = np.einsum("ab,bj->aj",np.diag(s),vd)

    for nn in range(2,nspins):
        print(f"Factorizing {nn}th tensor")
        #print(f"{np.shape(r)=} ")
        s_vd = np.reshape(s_vd,(virtual_dim*2,2**(nspins-nn)))
        #print(f"{np.shape(r)=}")
        u, s, vd = svd_trunc(s_vd)
        mps_tensors.append(np.reshape(u,(virtual_dim,2,np.shape(u)[1])))
        virtual_dim = np.shape(u)[1]
        print(f"{virtual_dim=}")
        s_vd = np.einsum("ab,bj->aj",np.diag(s),vd)


    mps_tensors.append(np.reshape(s_vd, (virtual_dim, 2)))


    return mps_tensors

In [218]:
w = (np.random.rand(1024) + 1j*np.random.rand(1024) )
w = w/np.linalg.norm(w)

mpsten = tensor_to_mps_svd_trunc(w)

len(mpsten)

for ii,ten in enumerate(mpsten):
    print(f"{ii} - {np.shape(ten)}")

w_reconstructed = wten_from_mps(mpsten)

np.allclose(w, w_reconstructed)

nspins=10
cut=2
Last SV = 0.3572190917265257
virtual_dim=2
Factorizing 2th tensor
cut=4
Last SV = 0.23408990890403156
virtual_dim=4
Factorizing 3th tensor
cut=8
Last SV = 0.14124699061367757
virtual_dim=8
Factorizing 4th tensor
cut=16
Last SV = 0.06882410114321412
virtual_dim=16
Factorizing 5th tensor
cut=32
Last SV = 0.00429207758695352
virtual_dim=32
Factorizing 6th tensor
cut=16
Last SV = 0.06623203191747376
virtual_dim=16
Factorizing 7th tensor
cut=8
Last SV = 0.14327041528557605
virtual_dim=8
Factorizing 8th tensor
cut=4
Last SV = 0.23992763723592606
virtual_dim=4
Factorizing 9th tensor
cut=2
Last SV = 0.3539823998818235
virtual_dim=2
0 - (2, 2)
1 - (2, 2, 4)
2 - (4, 2, 8)
3 - (8, 2, 16)
4 - (16, 2, 32)
5 - (32, 2, 16)
6 - (16, 2, 8)
7 - (8, 2, 4)
8 - (4, 2, 2)
9 - (2, 2)
1 - (2, 2) * (2, 2, 4)
2 - (4, 4) * (4, 2, 8)
3 - (8, 8) * (8, 2, 16)
4 - (16, 16) * (16, 2, 32)
5 - (32, 32) * (32, 2, 16)
6 - (64, 16) * (16, 2, 8)
7 - (128, 8) * (8, 2, 4)
8 - (256, 4) * (4, 2, 2)


True

For a random state, chances are that - unless we truncate very aggressively - we don't gain much. 

But let's try with a different state with more structure

In [219]:
def build_wstate(L):
    up = np.array([[1],[0]])
    do = np.array([[0],[1]])

    all_wnn = []

    wnn = up
    for jj in range(1,L):
        wnn = np.kron(wnn, do)

    print(np.shape(wnn))

    all_wnn.append(wnn)

    for nn in range(1,L):
        wnn = do
        for jj in range(1,L):
            if jj == nn:
                wnn = np.kron(wnn, do)
            else:
                wnn = np.kron(wnn, up)
        
        all_wnn.append(wnn)

    wstate = sum(all_wnn)/np.sqrt(L)
    return wstate.T

First factorize it with QR 

In [220]:
wstate = build_wstate(10)
print(f"{np.linalg.norm(wstate)}")

mpsten = tensor_to_mps_qr(wstate)

len(mpsten)

for ii,ten in enumerate(mpsten):
    print(f"{ii} - {np.shape(ten)}")

w_reconstructed = wten_from_mps(mpsten)
print(f"{np.linalg.norm(w_reconstructed)}")

np.allclose(wstate,w_reconstructed)

(1024, 1)
1.0
nspins=10
virtual_dim=2
Factorizing 2th tensor
virtual_dim=4
Factorizing 3th tensor
virtual_dim=8
Factorizing 4th tensor
virtual_dim=16
Factorizing 5th tensor
virtual_dim=32
Factorizing 6th tensor
virtual_dim=16
Factorizing 7th tensor
virtual_dim=8
Factorizing 8th tensor
virtual_dim=4
Factorizing 9th tensor
virtual_dim=2
0 - (2, 2)
1 - (2, 2, 4)
2 - (4, 2, 8)
3 - (8, 2, 16)
4 - (16, 2, 32)
5 - (32, 2, 16)
6 - (16, 2, 8)
7 - (8, 2, 4)
8 - (4, 2, 2)
9 - (2, 2)
1 - (2, 2) * (2, 2, 4)
2 - (4, 4) * (4, 2, 8)
3 - (8, 8) * (8, 2, 16)
4 - (16, 16) * (16, 2, 32)
5 - (32, 32) * (32, 2, 16)
6 - (64, 16) * (16, 2, 8)
7 - (128, 8) * (8, 2, 4)
8 - (256, 4) * (4, 2, 2)
0.9999999999999997


True

In [223]:
wstate = build_wstate(15)
print(f"{np.linalg.norm(wstate)}")

mpsten = tensor_to_mps_svd_trunc(wstate)

len(mpsten)

for ii,ten in enumerate(mpsten):
    print(f"{ii} - {np.shape(ten)}")

w_reconstructed = wten_from_mps(mpsten)
print(f"{np.linalg.norm(w_reconstructed)}")

np.allclose(wstate,w_reconstructed)

(32768, 1)
0.9999999999999999
nspins=15
cut=2
Last SV = 0.25819888974716115
virtual_dim=2
Factorizing 2th tensor
cut=3
Last SV = 0.0
virtual_dim=3
Factorizing 3th tensor
cut=3
Last SV = 0.0
virtual_dim=3
Factorizing 4th tensor
cut=3
Last SV = 0.0
virtual_dim=3
Factorizing 5th tensor
cut=3
Last SV = 6.092980223482375e-33
virtual_dim=3
Factorizing 6th tensor
cut=3
Last SV = 1.8025660961824373e-34
virtual_dim=3
Factorizing 7th tensor
cut=3
Last SV = 0.0
virtual_dim=3
Factorizing 8th tensor
cut=3
Last SV = 8.11640980363574e-49
virtual_dim=3
Factorizing 9th tensor
cut=3
Last SV = 2.342480633869552e-37
virtual_dim=3
Factorizing 10th tensor
cut=3
Last SV = 2.0680869533763253e-35
virtual_dim=3
Factorizing 11th tensor
cut=3
Last SV = 8.48785042906965e-30
virtual_dim=3
Factorizing 12th tensor
cut=3
Last SV = 0.0
virtual_dim=3
Factorizing 13th tensor
cut=3
Last SV = 1.5081580146586079e-16
virtual_dim=3
Factorizing 14th tensor
cut=2
Last SV = 0.3651483716701108
virtual_dim=2
0 - (2, 2)
1 - (2, 2, 

True

In [224]:

for ii,ten in enumerate(mpsten):
    print(f"{ii} - {np.shape(ten)}")

0 - (2, 2)
1 - (2, 2, 3)
2 - (3, 2, 3)
3 - (3, 2, 3)
4 - (3, 2, 3)
5 - (3, 2, 3)
6 - (3, 2, 3)
7 - (3, 2, 3)
8 - (3, 2, 3)
9 - (3, 2, 3)
10 - (3, 2, 3)
11 - (3, 2, 3)
12 - (3, 2, 3)
13 - (3, 2, 2)
14 - (2, 2)
